# [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-3.0-0075ff)](https://stablediffusion.vn/update/) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
🧨 `Support Lora Flux, Kontext, Qwen, Hi-Dream, Wan`

#☕️ 1.Cài đặt

In [ ]:
#@title ✨ Cài đặt UI

root_dir = "/content"
repo_dir = f"{root_dir}/SDVN-training-colab-flux"
toolkit_dir = f"{repo_dir}/ai-toolkit"
config_dir = f"{repo_dir}/Config"
data_dir = f"/content/drive/MyDrive/SD-Data"

UI_Train =  "Colab" # @param ["Colab","Toolkit_UI"]

%cd /content
from google.colab import drive
drive.mount('/content/drive')
import os
if not os.path.exists(repo_dir):
  !git clone  --recurse-submodules https://github.com/StableDiffusionVN/SDVN-training-colab-flux {repo_dir}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={repo_dir}/libtcmalloc_minimal.so.4

%cd {toolkit_dir}
!git checkout main
!git pull
!pip install -q $(sed -n '5p' {repo_dir}/ai-toolkit/requirements.txt)
!pip install -q -r {repo_dir}/requirements.txt
%run {repo_dir}/TrainScript.ipynb

if UI_Train == "Toolkit_UI":
  !wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  %cd /content/SDVN-training-colab-flux/ai-toolkit/ui
  sever_flare(8675)
  !npm run build_and_start

#⏳ 2. Cấu hình train

In [ ]:
#@title ✨ 2.1 Xử lý dữ liệu

# @markdown 🟡 `Train nhiều lora bằng cách nhập nhiều thư mục, cách nhau bằng dấu phẩy (,)`
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
# @markdown 🟡 `Thư mục control chỉ áp dụng cho Flux Kontext (Không bắt buộc)`

# @markdown 💡 `Ảnh của thư mục Control được coi là ảnh đầu vào, thư mục train được coi là ảnh kết quả, kontext có thể train không cần control folder. 2 thư mục cần trùng tên ảnh`
ControlFolder = ""  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

#@markdown 💡 ` Florence yêu cầu khởi động lại phiên (Ctr + M + .) và chạy lại từ đầu, nên dùng Gemini API thay thế hoặc dùng Caption_tools riêng`

#@markdown [![](https://img.shields.io/badge/Caption-Tools-blue)](https://colab.research.google.com/github/StableDiffusionVN/SDVN-training-colab-flux/blob/main/SDVN_Caption_Tools.ipynb)

Caption = 'None' # @param ['None', 'WD14','Florence', 'APIGemini | 2.5 Flash', ' APIGemini | 2.5 Pro', 'APIGemini | 2.5 Flash Lite',  'APIOpenAI | GPT 5', 'APIOpenAI | GPT 5-mini', 'APIOpenAI | GPT 5-nano']
APIkey = "" # @param {type:'string'}
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption = 'None'
  Custom_Caption = ''
  AddFolderName = False
extension =".txt"
train_folder_list = TrainFolder.split(",")
control_folder_list = ControlFolder.split(",")

Cap_prompt = {
    'Short':['<CAPTION>',10,30,'short'],
    'Medium':['<DETAILED_CAPTION>',10,100,'medium-length'],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,'very long']
}
  
for folder in train_folder_list:
  if DataClean == True :
    %cd /content
    clean_directory(folder)
  if Caption != "None":
    caption_dir(folder,Cap_prompt[Caption_Length][0])
  if AddFolderName:
    add_forder_name(folder)
  if Custom_Caption != "":
    process_dir(folder, Custom_Caption, Append, Remove_Caption)

In [ ]:
#@title ⚙️ 2.2 Cài đặt train
TypeTrain = 'Flux Kontext' # @param ['Flux','Flux Kontext', "Qwen", "Qwen-Edit", "Hi-Dream", "Hi-Dream E1",  "Wan2.1 I2V (14B-720P)",  "Wan2.1 I2V (14B-480P)", "Wan2.1 (14B)", "Wan2.2 TI2V 5B", "Wan2.2 (14B)", "Wan2.2 I2V (14B)" ]
Low_VRAM = False #@param {type:"boolean"}
# @markdown 🟡 `Đặt tên không có kỹ tự trống hoặc ký tự đặc biệt`
Lora_name = "lora_name" # @param {"type":"string","placeholder":"Tên lora"}
OutputFolder = "/content/drive/MyDrive/SD-Data/Lora" #@param {type:'string'}
# @markdown 🟡 `Điều chỉnh steps theo mức độ train mong muốn, các thư mục đặt tên theo cú pháp {steps}_{name} sẽ tự nhận steps riêng theo tên`
Steps = 1000 #@param {type:'number'}
Save_steps = 200 #@param {type:'number'}
# @markdown 🟡 `Đặt = 1 nếu train kontext với controlfolder`
Batch_size = 1 #@param {type:'number'}
# @markdown 🟡 `Các thông số nên để theo mặc định`
Lr = 1e-4 #@param {type:'number'}
Dim = 16 #@param {type:'number'}
Alpha = 16 #@param {type:'number'}
# @markdown 🟡 `Có thể train nhiều size VD: 512,768,1024`
Resolution = "1024" #@param {type:'string'}
Train_TexEncoder = False
Sampler_Prompt = "" #@param {type:'string'}
Control_Img_Path = "" #@param {type:'string'}
Sampler_Steps = 100 #@param {type:'number'}

folder_train = []
for folder in train_folder_list:
  if len(get_supported_images(folder)) > 0:
    folder_dic = {
    "path": folder,
    "control_path": control_folder_list[train_folder_list.index(folder)] if ControlFolder != "" else "",
    "steps": get_steps(folder)[0],
    "name": get_steps(folder)[1].replace(" ", "_")
    }
    folder_train.append(folder_dic)
  else:
    print(f"Thư mục [ {folder} ] có thể không chứa ảnh được hỗ trợ, hãy kiểm tra lại (.png, .jpg, .jpeg, .webp, .bmp, .JPG, .JPEG, .PNG)")

for folder in folder_train:
    config(folder)

#3. 🧨 Train

In [ ]:
#@title ⏳ Run Lora Train
AutoDisconect = False #@param {type:"boolean"}
train()
if AutoDisconect:
    from time import sleep
    sleep(180)
    from google.colab import runtime
    runtime.unassign()